In [60]:
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np

# --- Fechas ---
weeks = 56


In [61]:
import xmlrpc.client


# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')
df_merged["product_id"] = df_merged["product_id"].astype(str)

# Guardar
df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos
#para 105 semanas se demora aproximadamente 14 minutos

Facturas encontradas: 55346
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 200000 registros de líneas...
Descargados 215399 registros de líneas...
Total de líneas descargadas: 215399
Total de líneas descargadas con información de factura: 215399


In [62]:
df = df_merged.copy()

In [63]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["product_nom"] = df["product_name"].str.extract(r"\]\s*([A-Z]\w+)")


df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [64]:
df.head()

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
0,"[17208, '[DAB02856025] DA2856 FILTRO AIRE KOMA...",81200.0,1638815,1.0,"[DAB02856025] DA2856 FILTRO AIRE KOMATSU, J.DE...",FBUC6692,2025-10-20 19:50:06,FBUC6692,444158,SUCURSAL BUCARAMANGA,FABIAN BLANCO,SUCURSALES / Sucursal Bucaramanga / EXTERNOS B...,45554,[CO9002029679] INGENIERIA Y MINERIA DE SANTAND...,VENDEDOR EXTERNO,INGENIERIA Y MINERIA DE SANTANDER S.A.S.,BD7,SUCURSAL BUCARAGAMNGA,DAB02856025,DA2856,10,2025,20,DONSSON
1,"[19181, '[BCS10014125] GS014A FILTRO COMBUST. ...",30800.0,1638813,1.0,[BCS10014125] GS014A FILTRO COMBUST. CUMMINS. ...,FBUC6692,2025-10-20 19:50:06,FBUC6692,444158,SUCURSAL BUCARAMANGA,FABIAN BLANCO,SUCURSALES / Sucursal Bucaramanga / EXTERNOS B...,45554,[CO9002029679] INGENIERIA Y MINERIA DE SANTAND...,VENDEDOR EXTERNO,INGENIERIA Y MINERIA DE SANTANDER S.A.S.,BD7,SUCURSAL BUCARAGAMNGA,BCS10014125,GS014A,10,2025,20,BALDWIN
2,"[17690, '[DAB04805025] DA4805 FILTRO AIRE JCB ...",36000.0,1638816,1.0,[DAB04805025] DA4805 FILTRO AIRE JCB 32/912902...,FBUC6692,2025-10-20 19:50:06,FBUC6692,444158,SUCURSAL BUCARAMANGA,FABIAN BLANCO,SUCURSALES / Sucursal Bucaramanga / EXTERNOS B...,45554,[CO9002029679] INGENIERIA Y MINERIA DE SANTAND...,VENDEDOR EXTERNO,INGENIERIA Y MINERIA DE SANTANDER S.A.S.,BD7,SUCURSAL BUCARAGAMNGA,DAB04805025,DA4805,10,2025,20,DONSSON
3,"[19534, '[BLS00111125] GS111 FILTRO ACEITE - C...",35100.0,1638814,1.0,[BLS00111125] GS111 FILTRO ACEITE - CUMMINS 6B...,FBUC6692,2025-10-20 19:50:06,FBUC6692,444158,SUCURSAL BUCARAMANGA,FABIAN BLANCO,SUCURSALES / Sucursal Bucaramanga / EXTERNOS B...,45554,[CO9002029679] INGENIERIA Y MINERIA DE SANTAND...,VENDEDOR EXTERNO,INGENIERIA Y MINERIA DE SANTANDER S.A.S.,BD7,SUCURSAL BUCARAGAMNGA,BLS00111125,GS111,10,2025,20,BALDWIN
4,"[49888, '[DAB12570025] DA2570UHE FILTRO DE AIR...",466800.0,1638812,12.0,[DAB12570025] DA2570UHE FILTRO DE AIRE ALTA E...,FCOT26787,2025-10-20 19:48:10,FCOT26787,444157,PRINCIPAL COTA,LUIS IGNACIO ARANGO PINZON,EXTERNOS,42697,[CO9014577015] STAR SUMINISTROS INTEGRALES S.A.S,VENDEDOR EXTERNO,STAR SUMINISTROS INTEGRALES S.A.S,BD1,PRINCIPAL COTA,DAB12570025,DA2570UHE,10,2025,20,DONSSON


## VENTAS PERDIDAS

In [65]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["fecha", "<=", FECHA_FIN], ["almacenamiento_tipo", "=", "agotado"]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

df_vp = pd.DataFrame(lost_sales_data)

# ===============================
# 4. Normalizar columnas
# ===============================

for col in ["store_id", "cliente_id"]:
    df_vp[col.replace('_id', '_name')] = df_vp[col].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else str(x) if x else None
    )
    df_vp[col + "_num"] = df_vp[col].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
    )

# Eliminar columnas originales
df_vp = df_vp.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# ===============================
# 4.1. Forzar tipos de columnas antes de exportar
# ===============================
for col in df_vp.columns:
    if col.endswith("_num"):
        df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").astype("Int64")
    else:
        df_vp[col] = df_vp[col].astype("string")

df_vp = df_vp.copy()



📅 Descargando registros desde 2024-09-23 hasta 2025-10-20


In [66]:
df_vp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73705 entries, 0 to 73704
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cantidad_reservada   73705 non-null  string
 1   product_ref          73705 non-null  string
 2   cantidad             73705 non-null  string
 3   almacenamiento_tipo  73705 non-null  string
 4   fecha                73705 non-null  string
 5   cantidad_existencia  73705 non-null  string
 6   origen               73705 non-null  string
 7   store_name           73705 non-null  string
 8   store_id_num         73705 non-null  Int64 
 9   cliente_name         73705 non-null  string
 10  cliente_id_num       73705 non-null  Int64 
dtypes: Int64(2), string(9)
memory usage: 6.3 MB


In [67]:
# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("M").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "mes", "ano", "Semana", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_month = lost_by_week


In [68]:
vp_month[vp_month["product_ref"]=="DAB08177025"]

,store_name,product_ref,mes,ano,Semana,dia,lost_sales,veces_vp
4060,PRINCIPAL COTA,DAB08177025,6,2025,2025-06-01,1,1.0,1
4061,PRINCIPAL COTA,DAB08177025,7,2025,2025-07-01,1,6.0,5
4062,PRINCIPAL COTA,DAB08177025,8,2025,2025-08-01,1,7.0,3
4063,PRINCIPAL COTA,DAB08177025,9,2025,2025-09-01,1,4.0,4
4064,PRINCIPAL COTA,DAB08177025,10,2025,2025-10-01,1,1.0,1
12251,SUCURSAL BARRANQUILLA,DAB08177025,9,2025,2025-09-01,1,6.0,1
12252,SUCURSAL BARRANQUILLA,DAB08177025,10,2024,2024-10-01,1,0.0,1
18431,SUCURSAL BUCARAMANGA,DAB08177025,2,2025,2025-02-01,1,0.0,1
18432,SUCURSAL BUCARAMANGA,DAB08177025,6,2025,2025-06-01,1,0.0,1
28005,SUCURSAL CALLE 6,DAB08177025,5,2025,2025-05-01,1,2.0,3


In [69]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
#df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")


df_vp.to_excel("/home/donsson/proyectos/VENTAS/prom_ventas/vp_mes.xlsx")

#Tiempo promedio 3 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20251020.xlsx


## VISUM

In [70]:
clientes = {"VISUM LTDA",
"VILMAR LUBRICANTES S.A.S.",
"ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.", 
"SODATRANS LTDA",
"ESTACION Y LUBRICANTES CASABLANCA S.A.S"}

fecha_inicial = "09/1/2024"
fecha_final = "09/30/2025"

df_h = df[(df["client_name"].isin(clientes)) & (df["date_invoice"] >= fecha_inicial) & (df["date_invoice"] <= fecha_final) & (df["salesperson_name"] == "YEIS OSWALDO MUÑOZ GUERRERO")  ]

df_h = df_h.drop(columns=["product_id","id_x","id_y","partner_id_num","client_name_inv","invoice_name","origen","number"])

In [71]:
df_h["date_invoice"].max()

Timestamp('2025-09-19 14:59:01')

In [72]:
df_h.sample(10)

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
138317,483600.0,6.0,[DAB02937025] DA2937 FILTRO AIRE HINO GD Y GH ...,2025-02-18 13:03:51,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB02937025,DA2937,2,2025,18,DONSSON
138195,85800.0,3.0,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2025-02-18 13:23:28,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES CASABLANCA S.A.S,PRINCIPAL COTA,DAB02570025,DA2570,2,2025,18,DONSSON
122306,104800.0,2.0,"[BHS00435125] GS435 FILTRO HIDRAULICO FORD, NE...",2025-03-17 20:54:59,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,BHS00435125,GS435,3,2025,17,BALDWIN
121263,534000.0,12.0,[DAB02547025] DA2547 FILTROAIRE 1_ DONSSON -K...,2025-03-19 13:39:25,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAB02547025,DA2547,3,2025,19,DONSSON
169110,155400.0,3.0,[BCE00609125] G609 FILTRO COMBUSTIBLE MERCEDES...,2024-12-18 13:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCE00609125,G609,12,2024,18,BALDWIN
100561,66000.0,2.0,[BCS00015125] GS015 FILTRO COMBUSTIBLE CUMMINS...,2025-04-25 21:05:27,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BCS00015125,GS015,4,2025,25,BALDWIN
192533,494900.0,7.0,"[BLS00136125] GS136 FILTRO ACEITE CUMMINS,CASE...",2024-11-06 14:12:53,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00136125,GS136,11,2024,6,BALDWIN
169092,416400.0,3.0,[BLS00289125] GS289 FILTRO SECADOR FRENOS MERC...,2024-12-18 13:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00289125,GS289,12,2024,18,BALDWIN
192507,249000.0,6.0,[BCS00025125] GS025 FILTRO COMBUSTIBLE CUMMINS...,2024-11-06 14:12:53,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCS00025125,GS025,11,2024,6,BALDWIN
68512,192600.0,6.0,[DAB04937025] DA4937 FILTRO AIRE 2_ HINO GD Y ...,2025-06-24 12:57:45,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAB04937025,DA4937,6,2025,24,DONSSON


In [73]:
df_h[df_h["client_name"]=="VILMAR LUBRICANTES S.A.S."]

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
36033,184800.0,3.0,[BLS00106125] GS106 FILTRO ACEITE FULL-FLOW MA...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00106125,GS106,8,2025,21,BALDWIN
36034,378000.0,6.0,[BLS00311125] GS311 FILTRO ACEITE BALDWIN - CU...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00311125,GS311,8,2025,21,BALDWIN
36035,724800.0,24.0,[DAB02788025] DA2788 FILTRO AIRE DONSSON - Ch...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB02788025,DA2788,8,2025,21,DONSSON
36036,49500.0,1.0,[DAB09037025] DA9037 FILTRO AIRE JOHN DEERE - ...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB09037025,DA9037,8,2025,21,DONSSON
36037,51000.0,1.0,[DAB09191025] DA9191 FILTRO SEGURIDAD FOTON AU...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB09191025,DA9191,8,2025,21,DONSSON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214962,59800.0,1.0,[BLS00475125] GS475 FILTRO ACEITE CATERPILLAR ...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00475125,GS475,9,2024,23,BALDWIN
214963,175200.0,6.0,[DAB04968025] DA4968 FILTRO AIRE 2_ HINO FC (0...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAB04968025,DA4968,9,2024,23,DONSSON
214964,54600.0,1.0,[BLS00646125] GS646 FILTRO ACEITE MAQUINARI CA...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00646125,GS646,9,2024,23,BALDWIN
214965,266400.0,6.0,[DAE04304025] DA4304 FILTRO AIRE 2_ CHEVROLET ...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAE04304025,DA4304,9,2024,23,DONSSON


In [74]:
#df_h.to_excel("harold_vendedor.xlsx")